In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import glob
import platform
import pathlib  
import os.path
import netCDF4 as nc
import geopandas as gpd
import rasterio
from rasterio.mask import mask

from superflexpy.implementation.elements.gr4j import BaseElement, UnitHydrograph1, UnitHydrograph2, FluxAggregator 
from superflexpy.implementation.elements.gr4j import InterceptionFilter, ProductionStore, RoutingStore
from superflexpy.implementation.elements.structure_elements import Transparent, Splitter, Junction
from superflexpy.implementation.root_finders.pegasus import PegasusPython
from superflexpy.implementation.numerical_approximators.implicit_euler import ImplicitEulerPython
from superflexpy.framework.unit import Unit
from superflexpy.framework.node import Node
from superflexpy.framework.network import Network
# from utils.precipitation_evaporation import *

In [3]:
cwd = pathlib.Path().resolve()
src = cwd.parent
data = src.parent.parent.parent
OS_type = platform.system()

if OS_type == 'Darwin':
    username = 'Mats '
    data_path = os.path.join(data, 'data_tana', 'catchments')
    shape_path = os.path.join(data, 'data_tana', 'catchments')
    
else:
    username = 'Mootje'
    data_path = os.path.join(data, 'OneDrive - Delft University of Technology', 'TU Delft', 'Master ENVM', 'MDP', 'Model', 'Data', 'Mini_data')
    # shape_path = os.path.join(data, 'OneDrive - Delft University of Technology', 'TU Delft', 'Master ENVM', 'MDP', 'Model', 'Data', 'Shapefiles','Mini_shapes')

print(f"Welcome {username}, have a wondeful day on your {OS_type} machine. Your data should be located in {data_path}")
print(data_path)

Welcome Mootje, have a wondeful day on your Windows machine. Your data should be located in C:\Users\Moniek van Zon\OneDrive - Delft University of Technology\TU Delft\Master ENVM\MDP\Model\Data\Mini_data
C:\Users\Moniek van Zon\OneDrive - Delft University of Technology\TU Delft\Master ENVM\MDP\Model\Data\Mini_data


In [4]:
data_files = glob.glob(os.path.join(data_path, '*.nc'))
print(len(data_files))
datasets = {}
file_identifier_list = []

for file_path in data_files:
    # Extract the file identifier from the file name
    file_name = os.path.basename(file_path)  # Get just the file name
    file_identifier = file_name.split('_')[1]
    file_identifier2 = file_identifier.split('.')[0]
    file_identifier_list.append(file_identifier2)
   
    # Open the dataset
    dataset = xr.open_dataset(file_path)
    # print(dataset)
    # Add the file identifier as a new coordinate
    dataset = dataset.assign_coords(file_identifier=file_identifier)
    
    # Add the dataset to the dictionary with the file identifier as the key
    datasets[file_identifier2] = dataset



263


In [5]:
def GR4J_element(x1,x2,x3,x4, data_set, file_name):
    #x1, x2, x3, x4 = (50.0, 0.1, 20.0, 3.5)

    # name = data_set[file_name]
    name = file_name
    # print(name)
    # name = name.replace('_', '-')  
    area = data_set[file_name]['area_m2'].values[0]
    # print(area)
    precipitation = data_set[file_name]['precipitation'].values
    # print(precipitation)
    evaporation =  data_set[file_name]['average_evap'].values
    # print(evaporation)

    root_finder = PegasusPython()  # Use the default parameters
    numerical_approximation = ImplicitEulerPython(root_finder)

    interception_filter = InterceptionFilter(id='ir')

    production_store = ProductionStore(parameters={'x1': x1, 'alpha': 2.0,
                                                   'beta': 5.0, 'ni': 4/9},
                                       states={'S0': 5.0},
                                       approximation=numerical_approximation,
                                       id='ps')

    splitter = Splitter(weight=[[0.9], [0.1]],
                        direction=[[0], [0]],
                        id='spl')

    unit_hydrograph_1 = UnitHydrograph1(parameters={'lag-time': x4},
                                        states={'lag': None},
                                        id='uh1')

    unit_hydrograph_2 = UnitHydrograph2(parameters={'lag-time': 2*x4},
                                        states={'lag': None},
                                        id='uh2')

    routing_store = RoutingStore(parameters={'x2': x2, 'x3': x3,
                                             'gamma': 5.0, 'omega': 3.5},
                                 states={'S0': 20.0},
                                 approximation=numerical_approximation,
                                 id='rs')

    transparent = Transparent(id='tr')

    junction = Junction(direction=[[0, None],  # First output
                                   [1, None],  # Second output
                                   [None, 0]], # Third output
                        id='jun')

    flux_aggregator = FluxAggregator(id='fa')

    catchment_element = Unit(layers=[[interception_filter],
                     [production_store],
                     [splitter],
                     [unit_hydrograph_1, unit_hydrograph_2],
                     [routing_store, transparent],
                     [junction],
                     [flux_aggregator]],
             id=f'unit-{name}')
    
    
    current_node = Node(
    units=[catchment_element],
    weights = [1.0],
    area=area,
    id=f'node-{name}'
    )
    
    current_node.set_input([evaporation, precipitation])
    
    return current_node

In [6]:
x1, x2, x3, x4 = 50.0, 0.1, 20., 2.4

In [7]:
node_1 = GR4J_element(x1, x2, x3, x4, datasets, '1021')
node_2 = GR4J_element(x1, x2, x3, x4, datasets, '632')
node_3 = GR4J_element(x1, x2, x3, x4, datasets, '144')


In [16]:
nodes_dict = {}

# Iterate over each filename
for filename in file_identifier_list:
    # Call GR4J_element function for the current filename
    node = GR4J_element(x1, x2, x3, x4, datasets, filename)
    # Add the Node object to the dictionary with filename as key
    nodes_dict[filename] = node

Module: superflexPy
Node: node-1021
Units:
	['unit-1021']
Weights:
	[1.0]
********************
********************
Module: superflexPy
Unit: unit-1021
Layers:
	[['ir'], ['ps'], ['spl'], ['uh1', 'uh2'], ['rs', 'tr'], ['jun'], ['fa']]
********************
Module: superflexPy
Element: ir

********************
Module: superflexPy
Element: ps
Parameters:
	unit-1021_ps_x1 : 50.0
	unit-1021_ps_alpha : 2.0
	unit-1021_ps_beta : 5.0
	unit-1021_ps_ni : 0.4444444444444444
States:
	node-1021_unit-1021_ps_S0 : 5.0

********************
Module: superflexPy
Element: spl
Weight:
	[[0.9], [0.1]]
Direction:
	[[0], [0]]

********************
Module: superflexPy
Element: uh1
Parameters:
	unit-1021_uh1_lag-time : 2.4
States:
	node-1021_unit-1021_uh1_lag : None

********************
Module: superflexPy
Element: uh2
Parameters:
	unit-1021_uh2_lag-time : 4.8
States:
	node-1021_unit-1021_uh2_lag : None

********************
Module: superflexPy
Element: rs
Parameters:
	unit-1021_rs_x2 : 0.1
	unit-1021_rs_x3 : 2